In [ ]:
import ee

ee.Initialize()

# Today
now = ee.Date(Date.now())
# Year to be displayed in Title
present_year = now.get('year')
# 6 Months before Today
start_date = now.advance(-6, 'month')

# Clear default UI to use custom layout and widgets
ui.root.clear()

title = ui.Label('Tamil Nadu Landuse/Landcover Classification', {
    'stretch': 'horizontal',
    'textAlign': 'center',
    'fontWeight': 'bold',
    'fontSize': '24px'
})

present_year.evaluate(lambda value: title.setValue('Tamil Nadu Landuse/Landcover Classification - ' + str(value)))

def map_properties(map):
    """Changes the properties and style of the Map"""
    styledMapType = {
        'Grey': [
            {'stylers': [{'saturation': -80}]},
            {'featureType': 'road', 'elementType': 'geometry', 'stylers': [{'lightness': 100}, {'visibility': 'simplified'}]},
            {'featureType': 'road', 'elementType': 'labels'},
            {'featureType': 'water', 'stylers': [{'visibility': 'simplified'}]},
            {'featureType': "administrative", 'elementType': "geometry", 'stylers': [{'visibility': 'simplified'}]}
        ]
    }
    map.setOptions('Grey', styledMapType)
    map.setControlVisibility({
        'all': False,
        'zoomControl': True,
        'scaleControl': True,
    })
    return map

def maskS2clouds(image):
    """Removal of clouds by using the 'QA60' band"""
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

state_map = ui.Map()
map_properties(state_map)
state_map.addLayer(districts, {'color': 'black'}, 'boundary')
state_map.centerObject(districts)
# Welcome Screen - Title and styled map with TN districts layer and District Selector
ui.root.widgets().reset([title, state_map])
ui.root.setLayout(ui.Panel.Layout.Flow('vertical'))

def calNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def calNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Training the Random Forest Classifier with Sentinel2 image
newfc = vegetation.merge(waterbody).merge(builtup).merge(wasteland)
bands = ['B4', 'B3', 'B2', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'NDVI', 'NDWI']
train_image = calNDVI(train_image)
train_image = calNDWI(train_image)
training = train_image.select(bands).sampleRegions({
    'collection': newfc,
    'properties': ['class'],
    'scale': 30,
    'tileScale': 16,
})
classifier = ee.Classifier.smileRandomForest(10).train({
    'features': training,
    'classProperty': 'class',
    'inputProperties': bands,
})

def calculation(district):
    """RF Classification, Statistics and Widgets"""
    district_label = ui.Label(district, {
        'fontWeight':
